In [1]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual=[]
        self.convergence = []
        self.optimizer=""
        self.objfname=""
        self.startTime=0
        self.endTime=0
        self.executionTime=0
        self.lb=0
        self.ub=0
        self.dim=0
        self.popnum=0
        self.maxiers=0
        self.no_of_iterations=0
        self.positions=[]

In [2]:
def simplebounds(s,Lb,Ub):
    #% Apply the lower bound
    ns_tmp=np.array(s)
    I=ns_tmp<Lb
    ns_tmp[I]=Lb
  
    #% Apply the upper bounds 
    J=ns_tmp>Ub
    ns_tmp[J]=Ub
    #% Update this new move 
    return ns_tmp

In [4]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [5]:
def F1(x):
    s=numpy.sum(x**2);
    return s

In [355]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016
@author: Hossam Faris
"""

import random
import numpy
import math
#from solution import solution
import time


    

def bGWO(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=
    ggnn=0
    levy=np.zeros(SearchAgents_no)
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")
    
    #Initialize the positions of search agents
    Positions=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
    positions = numpy.random.choice([0, 1], size=(SearchAgents_no,dim))
    
    Convergence_curve=numpy.zeros(Max_iter)
    sol=solution()

     # Loop counter
    #print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    sol.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            Positions[i,:]=numpy.clip(Positions[i,:], lb, ub)

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        z=numpy.random.uniform()
        z=4*z*(1-z)
        
        a=2-l*((2)/Max_iter)+z; # a decreases linearly fron 2 to 0
        
        beta=0.5
        sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                
            s=Positions[i,:]
            
            u=numpy.random.uniform(size=len(s))*sigma
            v=numpy.random.uniform(size=len(s))
            step=u*abs(v)**(1/beta)
            stepsize=0.1*step*(s-Alpha_pos) #s-alpha_pos
            s=s+stepsize*numpy.random.uniform(size=len(s))
            Positions[i]=simplebounds(s,lb,ub)
            rand=random.random()
            if(sigmoid(Positions[i-1,j])>=rand):
                positions[i,j]=1

            else:
                positions[i,j]=0
        Convergence_curve[l]=Alpha_score
       
        """if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);"""
    
    timerEnd=time.time()  
    sol.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    sol.executionTime=timerEnd-timerStart
    sol.convergence=Convergence_curve
    sol.optimizer="GWO"
    sol.objfname=objf.__name__
    sol.no_of_iterations=l
    sol.positions=positions
    
    
    return sol
  
  

In [360]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016
@author: Hossam Faris
"""

import random
import numpy
import math
#from solution import solution
import time


    

def bGWO2(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=
    ggnn=0
    levy=np.zeros(SearchAgents_no)
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")
    
    #Initialize the positions of search agents
    Positions=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
    positions = numpy.random.choice([0, 1], size=(SearchAgents_no,dim))
    
    Convergence_curve=numpy.zeros(Max_iter)
    sol=solution()

    # Loop counter
    #print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    sol.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            Positions[i,:]=numpy.clip(Positions[i,:], lb, ub)

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        z=numpy.random.uniform()
        z=4*z*(1-z)
        
        a=2-l*((2)/Max_iter)+z; # a decreases linearly fron 2 to 0
        
        beta=0.5
        sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2
               
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                
            s=Positions[i,:]
            lamb=0.3
            
            u=numpy.random.uniform(size=len(s))*sigma
            v=numpy.random.uniform(size=len(s))
            step=u*abs(v)**(1/beta)
            stepsize=0.1*step*(s-Alpha_pos) #s-alpha_pos
            s=s+stepsize*numpy.random.uniform(size=len(s))
            Positions[i]=simplebounds(s,lb,ub)
            rand=random.random()
            if(Positions[i-1,j]>=lamb):
                positions[i,j]=1

            else:
                positions[i,j]=0
        Convergence_curve[l]=Alpha_score

        if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
    
    timerEnd=time.time()  
    sol.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    sol.executionTime=timerEnd-timerStart
    sol.convergence=Convergence_curve
    sol.optimizer="GWO"
    sol.objfname=objf.__name__
    sol.no_of_iterations=l
    sol.positions=positions
    
    
    return sol
  
  

In [6]:
import numpy as np
import pandas as pd

In [357]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.simplefilter('ignore')

clf=SVC(kernel='poly',degree=2,C=0.01)
clf6=KNeighborsClassifier(n_neighbors=5)


def findPositionArray2(dimension):
    egwo=bGWO2(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=dimension,SearchAgents_no=5)
    egwo.no_of_iterations,egwo.convergence[len(egwo.convergence)-1]
    newPos=egwo.positions[0]
    newnewPos=[]
    for i in range(len(egwo.positions[0])):
        if newPos[i]==0:
            newnewPos.append(False)
        else:
            newnewPos.append(True)
    return newnewPos 

def findPositionArray(dimension):
    egwo=bGWO(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=dimension,SearchAgents_no=5)
    egwo.no_of_iterations,egwo.convergence[len(egwo.convergence)-1]
    newPos=egwo.positions[0]
    newnewPos=[]
    for i in range(len(egwo.positions[0])):
        if newPos[i]==0:
            newnewPos.append(False)
        else:
            newnewPos.append(True)
    return newnewPos

def count(newnewPos):
    cou=0
    for i in newnewPos:
        if i==1:
            cou+=1
    return cou

def applyBinary(data,newnewPos):
    datanew=data.loc[:,newnewPos]
    return datanew

def SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all):
    clf.fit(x_train_all,y_train_all)
    ypredsvm=clf.predict(x_test_all)
    #print("all     :",clf.score(x_train_all,y_train_all),clf.score(x_test_all,y_test_all))
    return clf.score(x_test_all,y_test_all)
    
def SvmSel(clf,x_train,x_test,y_train,y_test):
    clf.fit(x_train,y_train)
    ypredsvm=clf.predict(x_test)
    #print("selected:",clf.score(x_train,y_train),clf.score(x_test,y_test))
    return clf.score(x_test,y_test)
    
def knnSel(clf6,x_train,x_test,y_train,y_test):
    clf6.fit(x_train,y_train)
    ypredKNN=clf6.predict(x_test)
    #print("selected:",clf6.score(x_train,y_train),clf6.score(x_test,y_test))
    return clf6.score(x_test,y_test)

def knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all):
    clf6.fit(x_train_all,y_train_all)
    ypredKNN=clf6.predict(x_test_all)
    #print("all     :",clf6.score(x_train_all,y_train_all),clf6.score(x_test_all,y_test_all))
    return clf6.score(x_test_all,y_test_all)

def function(dataset,name,clf=clf,clf6=clf6):
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    allData=pd.DataFrame(dataset.data)

    newnewPos=findPositionArray(len(dataset.data[0]))
    newnewPos2=findPositionArray2(len(dataset.data[0]))
    allnew=applyBinary(allData,newnewPos)
    allnew2=applyBinary(allData,newnewPos2)
    noOfFeatures=count(newnewPos)
    noOfFeatures2=count(newnewPos2)
    final=[]
    final.append(name)

    x_train,x_test,y_train,y_test=train_test_split(allnew.values,dataset.target,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(dataset.data,dataset.target,random_state=0)
    
    print("Bgwo1")
    print("total no of features=",len(newnewPos))
    final.append(len(newnewPos))
    final.append(noOfFeatures)
    print("no of selected features=",noOfFeatures)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    #print(svmSelected)
    final.append(sum(svmSelected)/10)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    #print(svmAll)
    final.append(sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(knnSelected)/10)
    #print(knnSelected)
    final.append(sum(knnSelected)/10)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(knnAll)/10)
    #print(knnAll)
    final.append(sum(knnAll)/10)
    
    
    x_train,x_test,y_train,y_test=train_test_split(allnew2.values,dataset.target,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(dataset.data,dataset.target,random_state=0)
    
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    print()
    print("Bgow2")
    print("total no of features=",len(newnewPos2))
    print("no of selected features=",noOfFeatures2)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    #print(svmSelected)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    #print(svmAll)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(knnSelected)/10)
    #print(knnSelected)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(knnAll)/10)
    #print(knnAll)
    
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(knnSelected)/10)
    final.append(sum(knnAll)/10)
    
    return final
    


def binary(ion,name,no,clf=clf,clf6=clf6):
    output=ion[ion.columns[no]].values
    del ion[ion.columns[no]]
    newnewPos=findPositionArray(len(ion.columns))
    newnewPos2=findPositionArray2(len(ion.columns))
    allnew=applyBinary(ion,newnewPos)
    allnew2=applyBinary(ion,newnewPos2)
    noOfFeatures=count(newnewPos)
    noOfFeatures2=count(newnewPos2)
    
    final=[]
    final.append(name)
    final.append(len(newnewPos))
    final.append(noOfFeatures)

    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    x_train,x_test,y_train,y_test=train_test_split(allnew.values,output,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(ion.values,output,random_state=0)

    print("Bgwo1")
    print("total no of features=",len(newnewPos))
    print("no of selected features=",noOfFeatures)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    #print(svmSelected)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(knnSelected)/10)
    #print(knnSelected)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(knnAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(knnSelected)/10)
    final.append(sum(knnAll)/10)
    
    
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    x_train,x_test,y_train,y_test=train_test_split(allnew2.values,output,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(ion.values,output,random_state=0)
    
    print()
    print("Bgow2")
    print("total no of features=",len(newnewPos2))
    print("no of selected features=",noOfFeatures2)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    #print(svmSelected)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(knnSelected)/10)
    #print(knnSelected)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(knnAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(knnSelected)/10)
    final.append(sum(knnAll)/10)
    
    return final

In [358]:
finalfinal=[]

In [361]:
from sklearn.datasets import load_breast_cancer,load_iris,load_wine,load_digits
breast=load_breast_cancer()

finalfinal.append(function(breast,"breast",clf6,clf6))


['At iteration 0 the best fitness is 80812.94728817645']
['At iteration 1 the best fitness is 43925.8091390922']
['At iteration 2 the best fitness is 43925.8091390922']
['At iteration 3 the best fitness is 43925.8091390922']
['At iteration 4 the best fitness is 43925.8091390922']
['At iteration 5 the best fitness is 43925.8091390922']
['At iteration 6 the best fitness is 38729.85410550015']
['At iteration 7 the best fitness is 38729.85410550015']
['At iteration 8 the best fitness is 38729.85410550015']
['At iteration 9 the best fitness is 38729.85410550015']
['At iteration 10 the best fitness is 38729.85410550015']
['At iteration 11 the best fitness is 38729.85410550015']
['At iteration 12 the best fitness is 38729.85410550015']
['At iteration 13 the best fitness is 38729.85410550015']
['At iteration 14 the best fitness is 38729.85410550015']
['At iteration 15 the best fitness is 38729.85410550015']
['At iteration 16 the best fitness is 38729.85410550015']
['At iteration 17 the best fi

['At iteration 224 the best fitness is 2.2857633319306427']
['At iteration 225 the best fitness is 2.2857633319306427']
['At iteration 226 the best fitness is 2.2857633319306427']
['At iteration 227 the best fitness is 2.2857633319306427']
['At iteration 228 the best fitness is 1.3187440910086403']
['At iteration 229 the best fitness is 1.2468279742643766']
['At iteration 230 the best fitness is 1.2468279742643766']
['At iteration 231 the best fitness is 1.2468279742643766']
['At iteration 232 the best fitness is 1.1544591018564798']
['At iteration 233 the best fitness is 1.1544591018564798']
['At iteration 234 the best fitness is 0.8138371589493408']
['At iteration 235 the best fitness is 0.8138371589493408']
['At iteration 236 the best fitness is 0.8138371589493408']
['At iteration 237 the best fitness is 0.8138371589493408']
['At iteration 238 the best fitness is 0.8138371589493408']
['At iteration 239 the best fitness is 0.8138371589493408']
['At iteration 240 the best fitness is 0

['At iteration 459 the best fitness is 2.2377877338697878e-11']
['At iteration 460 the best fitness is 2.2377877338697878e-11']
['At iteration 461 the best fitness is 2.2377877338697878e-11']
['At iteration 462 the best fitness is 1.6218508202202994e-11']
['At iteration 463 the best fitness is 1.6037444308926376e-11']
['At iteration 464 the best fitness is 1.6037444308926376e-11']
['At iteration 465 the best fitness is 1.1511610053582868e-11']
['At iteration 466 the best fitness is 1.0348787539705579e-11']
['At iteration 467 the best fitness is 1.0348787539705579e-11']
['At iteration 468 the best fitness is 1.0348787539705579e-11']
['At iteration 469 the best fitness is 1.0348787539705579e-11']
['At iteration 470 the best fitness is 1.0348787539705579e-11']
['At iteration 471 the best fitness is 8.027923027556537e-12']
['At iteration 472 the best fitness is 7.828633403058377e-12']
['At iteration 473 the best fitness is 5.417618996064248e-12']
['At iteration 474 the best fitness is 5.41

['At iteration 705 the best fitness is 3.0848152130404585e-22']
['At iteration 706 the best fitness is 3.0848152130404585e-22']
['At iteration 707 the best fitness is 2.5747197846710214e-22']
['At iteration 708 the best fitness is 2.512475626476255e-22']
['At iteration 709 the best fitness is 2.512475626476255e-22']
['At iteration 710 the best fitness is 2.512475626476255e-22']
['At iteration 711 the best fitness is 2.136021789429355e-22']
['At iteration 712 the best fitness is 2.136021789429355e-22']
['At iteration 713 the best fitness is 2.0343006555601517e-22']
['At iteration 714 the best fitness is 2.0112955037002524e-22']
['At iteration 715 the best fitness is 2.0112955037002524e-22']
['At iteration 716 the best fitness is 1.782521494957562e-22']
['At iteration 717 the best fitness is 1.6898243297992743e-22']
['At iteration 718 the best fitness is 1.5250758124832311e-22']
['At iteration 719 the best fitness is 1.5250758124832311e-22']
['At iteration 720 the best fitness is 1.52507

['At iteration 943 the best fitness is 1.2384098016541032e-27']
['At iteration 944 the best fitness is 1.2004558682275036e-27']
['At iteration 945 the best fitness is 1.1212207580560524e-27']
['At iteration 946 the best fitness is 1.1212207580560524e-27']
['At iteration 947 the best fitness is 1.0848595520509754e-27']
['At iteration 948 the best fitness is 1.0837574694319862e-27']
['At iteration 949 the best fitness is 1.0793448605639796e-27']
['At iteration 950 the best fitness is 1.0793448605639796e-27']
['At iteration 951 the best fitness is 1.0489908941119259e-27']
['At iteration 952 the best fitness is 1.0294912009770894e-27']
['At iteration 953 the best fitness is 1.0072851185286885e-27']
['At iteration 954 the best fitness is 1.0072851185286885e-27']
['At iteration 955 the best fitness is 1.0072851185286885e-27']
['At iteration 956 the best fitness is 9.266773975132416e-28']
['At iteration 957 the best fitness is 8.0566999896021075e-28']
['At iteration 958 the best fitness is 7.

In [306]:
iris=load_iris()

finalfinal.append(function(iris,"iris",clf6,clf6))


Bgwo1
total no of features= 4
no of selected features= 3
SVM
Selected 0.9736842105263157
[0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158]
All 0.9736842105263157
[0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158]
KNN
Selected 0.9736842105263157
[0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158]
All 0.9736842105263157
[0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158]

Bgow2
total no of features= 

In [307]:
wine=load_wine()

finalfinal.append(function(wine,"wine",clf6,clf6))


Bgwo1
total no of features= 13
no of selected features= 11
SVM
Selected 0.7333333333333333
[0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333]
All 0.7333333333333333
[0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333]
KNN
Selected 0.7333333333333333
[0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333]
All 0.7333333333333333
[0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333]

Bgow2
total no of features

In [308]:
digits=load_digits()
finalfinal.append(function(digits,"digits",clf6,clf6))

Bgwo1
total no of features= 64
no of selected features= 28
SVM
Selected 0.9399999999999997
[0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94]
All 0.9800000000000002
[0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98]
KNN
Selected 0.9399999999999997
[0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94, 0.94]
All 0.9800000000000002
[0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98]

Bgow2
total no of features= 64
no of selected features= 31
SVM
Selected 0.9333333333333333
[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]
All 0.9800000000000002
[0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98]
KNN
Selected 0.9333333333333333
[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.933333333333333

In [183]:
finalfinal

[['breast',
  30,
  13,
  0.9370629370629369,
  0.9370629370629369,
  0.9370629370629369,
  0.9370629370629369,
  0.9300699300699302,
  0.9370629370629369,
  0.9300699300699302,
  0.9370629370629369],
 ['iris',
  4,
  1,
  0.605263157894737,
  0.9736842105263157,
  0.605263157894737,
  0.9736842105263157,
  0.9473684210526316,
  0.9736842105263157,
  0.9473684210526316,
  0.9736842105263157],
 ['wine',
  13,
  6,
  0.7111111111111111,
  0.7333333333333333,
  0.7111111111111111,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333],
 ['digits',
  64,
  36,
  0.9622222222222222,
  0.9800000000000002,
  0.9622222222222222,
  0.9800000000000002,
  0.9777777777777776,
  0.9800000000000002,
  0.9777777777777776,
  0.9800000000000002]]

In [325]:
ion=pd.read_csv("ionosphere.data.csv")
finalfinal.append(binary(ion,"ion",34))

Bgwo1
total no of features= 34
no of selected features= 17
SVM
Selected 0.6363636363636365
[0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364]
All 0.6363636363636365
KNN
Selected 0.8977272727272727
[0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727, 0.8977272727272727]
All 0.909090909090909

Bgow2
total no of features= 34
no of selected features= 23
SVM
Selected 0.6363636363636365
[0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364, 0.6363636363636364]
All 0.6363636363636365
KNN
Selected 0.8863636363636365
[0.8863636363636364, 0.8863636363636364, 0.8863636363636364, 0.8863636363636364, 0

In [316]:
sonar=pd.read_csv("sonar.all-data.csv")
finalfinal.append(binary(sonar,"sonar",60))

Bgwo1
total no of features= 60
no of selected features= 29
SVM
Selected 0.5192307692307693
[0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693]
All 0.5192307692307693
KNN
Selected 0.7115384615384616
[0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616, 0.7115384615384616]
All 0.7115384615384616

Bgow2
total no of features= 60
no of selected features= 22
SVM
Selected 0.5192307692307693
[0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693, 0.5192307692307693]
All 0.5192307692307693
KNN
Selected 0.75
[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75]
All 0.7115384615384616


In [241]:
def f(x):
    if x=='x':
        return 1
    if x=='o':
        return -1
    if x=='b':
        return 0

In [333]:
tictactoe=pd.read_csv("tic-tac-toe.data.csv.csv")
coll=tictactoe.columns[:-1]
for i in coll:
    tictactoe[i+"-2"]=tictactoe[i].apply(f)
for i in coll:
    del tictactoe[i]
finalfinal.append(binary(tictactoe,"ticTacToe",0))

Bgwo1
total no of features= 9
no of selected features= 2
SVM
Selected 0.6416666666666667
[0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667]
All 0.6416666666666667
KNN
Selected 0.6333333333333332
[0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333, 0.6333333333333333]
All 1.0

Bgow2
total no of features= 9
no of selected features= 3
SVM
Selected 0.6416666666666667
[0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667, 0.6416666666666667]
All 0.6416666666666667
KNN
Selected 0.6208333333333335
[0.6208333333333333, 0.6208333333333333, 0.6208333333333333, 0.6208333333333333, 0.6208333333333333,

In [342]:
zoo=pd.read_csv("zoo.data.csv")
del zoo["aardvark"]
finalfinal.append(binary(zoo,"zoo",16))

Bgwo1
total no of features= 16
no of selected features= 9
SVM
Selected 0.5600000000000002
[0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56]
All 0.5600000000000002
KNN
Selected 0.7199999999999999
[0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.72, 0.72]
All 0.7999999999999999

Bgow2
total no of features= 16
no of selected features= 8
SVM
Selected 0.5600000000000002
[0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56]
All 0.5600000000000002
KNN
Selected 0.8400000000000001
[0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84]
All 0.7999999999999999


In [349]:
lymph=pd.read_csv("lymphography.data.csv")
finalfinal.append(binary(lymph,"lymphography",0))

Bgwo1
total no of features= 18
no of selected features= 9
SVM
Selected 0.5135135135135134
[0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135, 0.5135135135135135]
All 0.6486486486486488
KNN
Selected 0.7297297297297297
[0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297, 0.7297297297297297]
All 0.7567567567567569

Bgow2
total no of features= 18
no of selected features= 8
SVM
Selected 0.6486486486486488
[0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487, 0.6486486486486487]
All 0.6486486486486488
KNN
Selected 0.7837837837837839
[0.7837837837837838, 0.7837837837837838, 0.7837837837837838, 0.7837837837837838, 0.

In [353]:
heart=pd.read_csv("heart.dat.csv",delimiter=" ")
heart.shape
finalfinal.append(binary(heart,"heart",13))

Bgwo1
total no of features= 13
no of selected features= 6
SVM
Selected 0.75
[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75]
All 0.7647058823529411
KNN
Selected 0.5588235294117646
[0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647, 0.5588235294117647]
All 0.6029411764705881

Bgow2
total no of features= 13
no of selected features= 5
SVM
Selected 0.7205882352941176
[0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176, 0.7205882352941176]
All 0.7647058823529411
KNN
Selected 0.661764705882353
[0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529, 0.6617647058823529]
All 0.6029411764705881


In [279]:
def g(x):
    if x=='y':
        return 1
    if x=='n':
        return 0

In [287]:
congress=pd.read_csv("house-votes-84.data.csv")
congress
coll=congress.columns[1:]
for i in coll:
    congress[i+"-2"]=congress[i].apply(g)
for i in coll:
    del congress[i]
congress.dropna(inplace=True)
finalfinal.append(binary(congress,"congress",0))

Bgwo1
total no of features= 16
no of selected features= 9
SVM
Selected 0.48275862068965514
All 0.48275862068965514
KNN
Selected 0.48275862068965514
All 0.48275862068965514

Bgow2
total no of features= 16
no of selected features= 10
SVM
Selected 0.48275862068965514
All 0.48275862068965514
KNN
Selected 0.48275862068965514
All 0.48275862068965514


In [350]:
table=pd.DataFrame(finalfinal)

In [351]:
table.columns=["dataset","total features","Selected features","SVM on selected(bGwo1)","svm on all","Knn on selected","knn on all",
              "svm on selected(bGwo2)","svm on all","knn on selected","knn on all"]

In [352]:
table

,dataset,total features,Selected features,SVM on selected(bGwo1),svm on all,Knn on selected,knn on all,svm on selected(bGwo2),svm on all,knn on selected,knn on all
0,zoo,16,5,0.680000,0.560000,0.800000,0.800000,0.560000,0.560000,0.600000,0.800000
1,zoo,16,12,0.560000,0.560000,0.800000,0.800000,0.560000,0.560000,0.800000,0.800000
2,zoo,16,6,0.560000,0.560000,0.680000,0.800000,0.560000,0.560000,0.800000,0.800000
3,zoo,16,8,0.560000,0.560000,0.920000,0.800000,0.560000,0.560000,0.920000,0.800000
4,zoo,16,9,0.560000,0.560000,0.720000,0.800000,0.560000,0.560000,0.840000,0.800000
5,lymphography,18,7,0.621622,0.648649,0.729730,0.756757,0.648649,0.648649,0.702703,0.756757
6,lymphography,18,8,0.567568,0.648649,0.648649,0.756757,0.729730,0.648649,0.864865,0.756757
7,lymphography,18,13,0.648649,0.648649,0.783784,0.756757,0.621622,0.648649,0.648649,0.756757
8,lymphography,18,9,0.513514,0.648649,0.729730,0.756757,0.648649,0.648649,0.783784,0.756757


In [292]:
table.to_csv("datasets.csv",sep=",")

In [277]:
df=pd.read_csv("datasets.csv")
df

,Unnamed: 0,dataset,total features,Selected features,SVM on selected(bGwo1),svm on all,Knn on selected,knn on all,svm on selected(bGwo2),svm on all.1,knn on selected,knn on all.1
0,0,breast,30,18,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063
1,1,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.605263,0.973684,0.605263,0.973684
2,2,wine,13,9,0.733333,0.733333,0.733333,0.733333,0.800000,0.733333,0.800000,0.733333
3,3,digits,64,33,0.940000,0.980000,0.940000,0.980000,0.971111,0.980000,0.971111,0.980000
4,4,breast,30,16,0.916084,0.937063,0.916084,0.937063,0.951049,0.937063,0.951049,0.937063
5,5,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.921053,0.973684,0.921053,0.973684
6,6,wine,13,5,0.822222,0.733333,0.822222,0.733333,0.844444,0.733333,0.844444,0.733333
7,7,digits,64,32,0.933333,0.980000,0.933333,0.980000,0.975556,0.980000,0.975556,0.980000
8,8,ion,34,16,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364
9,9,ion,34,23,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364
